In [1]:
# Import Needed libraries
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score,StratifiedKFold,KFold
from sklearn.metrics import mean_absolute_error
import math
from sklearn.metrics import make_scorer
import csv
import numpy as np
import copy as cp
import matplotlib.pyplot as plt
from sklearn.multiclass import OneVsOneClassifier
import seaborn as sns
from typing import Tuple
from sklearn.model_selection import train_test_split
import fnmatch
import os
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_validate, StratifiedKFold, cross_val_predict


In [2]:
FILE_DIRECTORY = "/Users/sarahbanat/Desktop/Research/39BusWAttacks/"


In [3]:
import pandas as pd

# Adjust the path based on the location of your file in Google Drive
file_path = 'data/data.csv'
table = pd.read_csv(file_path)
#table.drop_duplicates(inplace=True)


In [4]:
# replace Nan with 0
table = table.fillna(0)

# drop the first row in the dataset since it is the only one with Fault_type =0 and Fault_location = 0
#table = table.drop(0)

#shuffle the data
table = shuffle(table, random_state=42)
table.reset_index(inplace=True, drop=True)

# print the columns names
column_names = list(table.columns.values)
print (column_names)


['flag', 'Normal_Mode_Case', 'Fault_Mode_Case', 'Fault_Type', 'Fault_Location', 'X', 'Attack_Type', 'Attack_Case', 'V1_Mag', 'V1_Ang', 'V1_zero', 'V1_F_Undershoot', 'V1_F_Overshoot', 'V1_F_OS_Time', 'V1_F_Sett_Time', 'V1_ROCOF', 'V1_F_Range', 'I1_Mag', 'I1_Ang', 'I1_zero', 'I1_F_Undershoot', 'I1_F_Overshoot', 'I1_F_OS_Time', 'I1_F_Sett_Time', 'I1_ROCOF', 'I1_F_Range', 'V2_Mag', 'V2_Ang', 'V2_zero', 'V2_F_Undershoot', 'V2_F_Overshoot', 'V2_F_OS_Time', 'V2_F_Sett_Time', 'V2_ROCOF', 'V2_F_Range', 'I2_Mag', 'I2_Ang', 'I2_zero', 'I2_F_Undershoot', 'I2_F_Overshoot', 'I2_F_OS_Time', 'I2_F_Sett_Time', 'I2_ROCOF', 'I2_F_Range', 'V3_Mag', 'V3_Ang', 'V3_zero', 'V3_F_Undershoot', 'V3_F_Overshoot', 'V3_F_OS_Time', 'V3_F_Sett_Time', 'V3_ROCOF', 'V3_F_Range', 'I3_Mag', 'I3_Ang', 'I3_zero', 'I3_F_Undershoot', 'I3_F_Overshoot', 'I3_F_OS_Time', 'I3_F_Sett_Time', 'I3_ROCOF', 'I3_F_Range', 'V4_Mag', 'V4_Ang', 'V4_zero', 'V4_F_Undershoot', 'V4_F_Overshoot', 'V4_F_OS_Time', 'V4_F_Sett_Time', 'V4_ROCOF', 'V4

In [5]:
table.head()


,flag,Normal_Mode_Case,Fault_Mode_Case,Fault_Type,Fault_Location,X,Attack_Type,Attack_Case,V1_Mag,V1_Ang,...,V39_F_Range,I39_Mag,I39_Ang,I39_zero,I39_F_Undershoot,I39_F_Overshoot,I39_F_OS_Time,I39_F_Sett_Time,I39_ROCOF,I39_F_Range
0,2,175,9,2,13,0.40,0,0,287322.8725,-13.844793,...,0.001706,2454.740111,-54.733797,1.195458e+03,59.372889,60.087647,0.154948,0.052865,14.965082,0.714759
1,2,722,8,8,14,0.35,0,0,287169.1217,-13.618481,...,0.001690,4220.254934,-75.890875,1.658273e+03,59.432451,60.124897,0.125521,0.113542,4.542180,0.692446
2,2,913,3,9,4,0.75,0,0,249626.7115,-17.765473,...,0.001762,2940.745195,-53.960762,8.298816e+02,59.047945,60.102524,0.099740,0.085677,21.247582,1.054579
3,2,496,2,7,5,0.55,0,0,277798.6729,-13.782176,...,0.001552,3259.395904,-44.270386,5.120703e-03,59.363201,60.126532,0.069531,0.080990,24.340793,0.763330
4,0,737,0,0,0,0.00,0,0,290076.7144,-13.883010,...,0.000775,2163.753662,-26.817703,2.520000e-07,59.997109,59.997886,0.000000,0.000260,0.000000,0.000777


In [6]:
print(column_names)


['flag', 'Normal_Mode_Case', 'Fault_Mode_Case', 'Fault_Type', 'Fault_Location', 'X', 'Attack_Type', 'Attack_Case', 'V1_Mag', 'V1_Ang', 'V1_zero', 'V1_F_Undershoot', 'V1_F_Overshoot', 'V1_F_OS_Time', 'V1_F_Sett_Time', 'V1_ROCOF', 'V1_F_Range', 'I1_Mag', 'I1_Ang', 'I1_zero', 'I1_F_Undershoot', 'I1_F_Overshoot', 'I1_F_OS_Time', 'I1_F_Sett_Time', 'I1_ROCOF', 'I1_F_Range', 'V2_Mag', 'V2_Ang', 'V2_zero', 'V2_F_Undershoot', 'V2_F_Overshoot', 'V2_F_OS_Time', 'V2_F_Sett_Time', 'V2_ROCOF', 'V2_F_Range', 'I2_Mag', 'I2_Ang', 'I2_zero', 'I2_F_Undershoot', 'I2_F_Overshoot', 'I2_F_OS_Time', 'I2_F_Sett_Time', 'I2_ROCOF', 'I2_F_Range', 'V3_Mag', 'V3_Ang', 'V3_zero', 'V3_F_Undershoot', 'V3_F_Overshoot', 'V3_F_OS_Time', 'V3_F_Sett_Time', 'V3_ROCOF', 'V3_F_Range', 'I3_Mag', 'I3_Ang', 'I3_zero', 'I3_F_Undershoot', 'I3_F_Overshoot', 'I3_F_OS_Time', 'I3_F_Sett_Time', 'I3_ROCOF', 'I3_F_Range', 'V4_Mag', 'V4_Ang', 'V4_zero', 'V4_F_Undershoot', 'V4_F_Overshoot', 'V4_F_OS_Time', 'V4_F_Sett_Time', 'V4_ROCOF', 'V4

In [7]:
######
needed_PMUs = [38] # use list(range(1,40)) for all PMUs. For specific PMUs use normal list ex. [1,3]
filtered_cols = np.empty([0], dtype=int)
######


In [8]:
for i in needed_PMUs:
  filtered_cols =  np.append(filtered_cols, fnmatch.filter(column_names, '?%s_*' % (i)))

fea_all = table[filtered_cols].copy()
################################
#fea_all = table.copy()
#fea_all.info()


# Pick the label
# column 0 flag (0->Normal, 1->Attack, 2->Fault)
# column 3 for Fault_Type label
# column 4 for Fault_Location Label
# column 5 for Distance Label (X)
# column 6 for Attack_Type label

label_fault_flag = table.iloc[: ,0].copy()
label_fault_type = table.iloc[: ,3].copy()
label_fault_location = table.iloc[: ,4].copy()
label_distance = table.iloc[: ,5].copy()
label_attack_type = table.iloc[: ,6].copy()


# Normalize the data
scaler = StandardScaler()
fea_all = scaler.fit_transform (fea_all)

X = fea_all
y_fault_flag = label_fault_flag
y_fault_type = label_fault_type
y_fault_location = label_fault_location
y_distance = label_distance
y_attack_type = label_attack_type


In [9]:
def classifaction_report_csv(report,writer):
    report_data = []
    lines = report.split('\n')
    for line in lines:
        row = []
        row_data = line.split('      ')
        for st in row_data:
          stripped = st.strip()
          #print(stripped)
          row.append(stripped)
        writer.writerow(row)


In [10]:
def write_results(label, scores, y, y_pred, exp_name, exp_num):
    file_name = FILE_DIRECTORY + label
    file_exists = os.path.isfile(file_name)

    with open(file_name, 'a', newline='') as file:
        writer = csv.writer(file)
        if not file_exists:
            writer.writerow(["Experiment Name"])
        writer.writerow([''])
        writer.writerow([''])
        writer.writerow(['-----------------------------------'])
        writer.writerow([''])
        writer.writerow([''])
        writer.writerow([exp_name])
        writer.writerow([f"Exp {exp_num}"])
        writer.writerow([str("Training Scores are: ")])
        writer.writerow([''])
        writer.writerow(scores['train_score'] * 100)
        writer.writerow([str("Average Training Score is: "), str(np.array(scores['train_score'].mean() * 100))])
        writer.writerow([''])
        writer.writerow([str("Testing Scores are: ")])
        writer.writerow(scores['test_score'] * 100)
        writer.writerow([str("Average Testing Score is: "), str(scores['test_score'].mean() * 100)])
        writer.writerow([''])
        writer.writerow([str("Classification Report for all data: ")])
        report = classification_report(y, y_pred)
        classifaction_report_csv(report, writer)
        matrix = confusion_matrix(y, y_pred)
        writer.writerow([''])
        writer.writerow([str("Confusion matrix, without normalization")])
        writer.writerows(matrix)
        matrix_normalized = confusion_matrix(y, y_pred, normalize='true')
        writer.writerow([str("Normalized confusion matrix")])
        writer.writerows(matrix_normalized)


In [11]:
# Generate an array with the needed features
column_names = list(table.columns.values)
exp_num = 1
exp_name = f"Experiment {exp_num}"


In [12]:
clf = LinearSVC(random_state = 42, max_iter=1200000)
stratifiedkf=StratifiedKFold(n_splits=10)

scores= cross_validate(clf, X, y_fault_flag, cv=stratifiedkf, return_train_score=True, return_estimator=True)
y_pred = cross_val_predict(clf, X, y_fault_flag, cv=stratifiedkf)
write_results ("results_fault_flag.csv",scores, y_fault_flag, y_pred, exp_name, exp_num)


/Users/sarahbanat/miniforge3/envs/research/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Users/sarahbanat/miniforge3/envs/research/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Users/sarahbanat/miniforge3/envs/research/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Users/sarahbanat/miniforge3/envs/research/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dua

# Filter data where FaultFlag=2 (Fault) for fault-related classifiers


In [13]:
# Filter data where FaultFlag=2 for fault classifiers and regressor
X_fault = fea_all[y_fault_flag == 2]
y_fault_type_filtered = y_fault_type[y_fault_flag == 2]
y_fault_location_filtered = y_fault_location[y_fault_flag == 2]
y_distance_filtered = y_distance[y_fault_flag == 2]


# Level 2: Classify Fault Type (for fault samples only)


In [14]:
clf = LinearSVC(random_state = 42, max_iter=1200000)
stratifiedkf=StratifiedKFold(n_splits=10)

scores= cross_validate(clf, X_fault, y_fault_type_filtered, cv=stratifiedkf, return_train_score=True, return_estimator=True)
y_pred = cross_val_predict(clf, X_fault, y_fault_type_filtered, cv=stratifiedkf)
write_results("results_fault_type.csv", scores, y_fault_type_filtered, y_pred, exp_name, exp_num)


/Users/sarahbanat/miniforge3/envs/research/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Users/sarahbanat/miniforge3/envs/research/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Users/sarahbanat/miniforge3/envs/research/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Users/sarahbanat/miniforge3/envs/research/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dua

# Level 3: Classify Fault Location (for fault samples only)


In [15]:
clf = LinearSVC(random_state = 42, max_iter=1200000)
stratifiedkf=StratifiedKFold(n_splits=10)

scores= cross_validate(clf, X_fault, y_fault_location_filtered, cv=stratifiedkf, return_train_score=True, return_estimator=True)
y_pred = cross_val_predict(clf, X_fault, y_fault_location_filtered, cv=stratifiedkf)
write_results("results_fault_location.csv", scores, y_fault_location_filtered, y_pred, exp_name, exp_num)

print(y_pred)
print(y_fault_location_filtered)


/Users/sarahbanat/miniforge3/envs/research/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Users/sarahbanat/miniforge3/envs/research/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Users/sarahbanat/miniforge3/envs/research/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Users/sarahbanat/miniforge3/envs/research/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dua

[13 13  4 ... 14 31 33]
0        13
1        14
2         4
3         5
7        32
         ..
17987    33
17988    10
17992    14
17994    30
17997    34
Name: Fault_Location, Length: 9000, dtype: int64


In [16]:
print(y_distance_filtered)


0        0.40
1        0.35
2        0.75
3        0.55
7        0.85
         ... 
17987    0.70
17988    0.15
17992    0.60
17994    0.65
17997    0.10
Name: X, Length: 9000, dtype: float64


# Level 4: Random Forest regression for the distance label (for fault samples only)


In [17]:
RF_reg = RandomForestRegressor(n_estimators=50,random_state=80)
kf=KFold(n_splits=10)

scores= cross_validate(RF_reg, X_fault, y_distance_filtered, cv=kf, return_train_score=True, return_estimator=True, scoring=( 'neg_mean_squared_error','neg_root_mean_squared_error','r2','neg_mean_absolute_error','neg_mean_absolute_percentage_error'))
y_pred = cross_val_predict(RF_reg, X_fault, y_distance_filtered, cv=kf)
np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})


In [18]:
file_name = FILE_DIRECTORY + "results_distance.csv"
file_exists = os.path.isfile(file_name)

with open(file_name, 'a', newline='') as file:
    writer = csv.writer(file)
    if not file_exists:
        writer.writerow(["Experiment Name"])
    writer.writerow([''])
    writer.writerow([''])
    writer.writerow(['-----------------------------------'])
    writer.writerow([''])
    writer.writerow([''])
    writer.writerow([exp_name])
    writer.writerow([f"Exp {exp_num}"])
    writer.writerow(["Train Mean squared error: "])
    writer.writerow(scores['train_neg_mean_squared_error'])
    writer.writerow(["Average Train Mean squared error: ", str(round(scores['train_neg_mean_squared_error'].mean(), 2))])
    writer.writerow([''])
    writer.writerow(["Train Root Mean squared error: "])
    writer.writerow(scores['train_neg_root_mean_squared_error'])
    writer.writerow(["Average Train Root Mean squared error: ", str(round(scores['train_neg_root_mean_squared_error'].mean(), 2))])
    writer.writerow([''])
    writer.writerow(["Train R2: "])
    writer.writerow(scores['train_r2'])
    writer.writerow(["Average Train R2: ", str(round(scores['train_r2'].mean(), 2))])
    writer.writerow([''])
    writer.writerow(["Train Mean absolute error: "])
    writer.writerow(scores['train_neg_mean_absolute_error'])
    writer.writerow(["Average Train Mean absolute error: ", str(round(scores['train_neg_mean_absolute_error'].mean(), 2))])
    writer.writerow([''])
    writer.writerow(["Train mean absolute percentage error: "])
    writer.writerow(scores['train_neg_mean_absolute_percentage_error'])
    writer.writerow(["Average Train mean absolute percentage error: ", str(round(scores['train_neg_mean_absolute_percentage_error'].mean(), 2))])
    writer.writerow([''])
    writer.writerow([''])
    writer.writerow(["Test Mean squared error: "])
    writer.writerow(scores['test_neg_mean_squared_error'])
    writer.writerow(["Average Test Mean squared error: ", str(round(scores['test_neg_mean_squared_error'].mean(), 2))])
    writer.writerow([''])
    writer.writerow(["Test Root Mean squared error: "])
    writer.writerow(scores['test_neg_root_mean_squared_error'])
    writer.writerow(["Average Test Root Mean squared error: ", str(round(scores['test_neg_root_mean_squared_error'].mean(), 2))])
    writer.writerow([''])
    writer.writerow(["Test R2: "])
    writer.writerow(scores['test_r2'])
    writer.writerow(["Average Test R2: ", str(round(scores['test_r2'].mean(), 2))])
    writer.writerow([''])
    writer.writerow(["Test Mean absolute error: "])
    writer.writerow(scores['test_neg_mean_absolute_error'])
    writer.writerow(["Average Test Mean absolute error: ", str(round(scores['test_neg_mean_absolute_error'].mean(), 2))])
    writer.writerow([''])
    writer.writerow(["Test mean absolute percentage error: "])
    writer.writerow(scores['test_neg_mean_absolute_percentage_error'])
    writer.writerow(["Average Test mean absolute percentage error: ", str(round(scores['test_neg_mean_absolute_percentage_error'].mean(), 2))])


# Optional: Level 5 - Classify Attack Type (for attack samples only)


In [19]:
# Filter data where FaultFlag=1 (Attack) for attack type classifier
X_attack = fea_all[y_fault_flag == 1]
y_attack_type_filtered = y_attack_type[y_fault_flag == 1]

# Check if there are multiple attack types
if len(np.unique(y_attack_type_filtered)) > 1:
    clf = LinearSVC(random_state = 42, max_iter=1200000)
    stratifiedkf=StratifiedKFold(n_splits=10)
    
    scores= cross_validate(clf, X_attack, y_attack_type_filtered, cv=stratifiedkf, return_train_score=True, return_estimator=True)
    y_pred = cross_val_predict(clf, X_attack, y_attack_type_filtered, cv=stratifiedkf)
    write_results("results_attack_type.csv", scores, y_attack_type_filtered, y_pred, exp_name, exp_num)
    print("Attack type classification completed")
else:
    print("Only one attack type found, skipping attack type classification")


/Users/sarahbanat/miniforge3/envs/research/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Users/sarahbanat/miniforge3/envs/research/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Users/sarahbanat/miniforge3/envs/research/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Users/sarahbanat/miniforge3/envs/research/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dua

Attack type classification completed


In [20]:
################################################################################
